In [2]:
import os
import cv2
import numpy as np
from keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore")
from keras.preprocessing.image import load_img, img_to_array 
from keras.models import  load_model
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# load model
model = load_model("final_model.keras")


face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [2]:
import cv2
import numpy as np
import tensorflow as tf
import statistics as st

# Initialize variables
i = 0
GR_dict = {0: (0, 255, 0), 1: (0, 0, 255)}

# Load the pre-trained emotion detection model
model = tf.keras.models.load_model('final_model.keras')

# Load the face cascade for detecting faces
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Prepare to collect predicted emotions
output = []

# Open the webcam (0 is the default camera, change if necessary)
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not access the camera.")
    exit()

# Maximum number of frames without faces detected before exiting
max_no_faces = 10
no_faces_count = 0

# Start capturing video frames
while i <= 50:
    ret, img = cap.read()  # Read frame from webcam

    if not ret:
        print("Failed to capture image")
        continue  # Skip this iteration and try again

    # Debug: Show the captured image to ensure the camera is working properly
    cv2.imshow("Captured", img)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.05, minNeighbors=5)

    if len(faces) == 0:
        no_faces_count += 1
        print("No faces detected")
        if no_faces_count >= max_no_faces:
            print("No faces detected for too many frames. Exiting.")
            break  # Exit the loop if no faces detected for too many frames
        continue  # Skip this iteration and try again

    # Reset the counter if faces are detected
    no_faces_count = 0

    # Process each face detected in the frame
    for x, y, w, h in faces:
        face_img = img[y:y + h, x:x + w]  # Crop the face region

        # Resize the face image to match model input size (96x96)
        resized = cv2.resize(face_img, (96, 96))
        
        # Normalize the image to match the model input range [0, 1]
        reshaped = resized.reshape(1, 96, 96, 3) / 255.0
        print(f"Input shape before prediction: {reshaped.shape}")

        try:
            # Get the emotion predictions from the model
            predictions = model.predict(reshaped)
            
            # Get the index of the highest prediction score
            max_index = np.argmax(predictions[0])

            # Emotion labels (should match your model's output classes)
            emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'neutral', 'surprise')
            predicted_emotion = emotions[max_index]
            output.append(predicted_emotion)

            # Draw a bounding box around the face and display the predicted emotion
            cv2.rectangle(img, (x, y), (x + w, y + h), GR_dict[1], 2)  # Face rectangle
            cv2.rectangle(img, (x, y - 40), (x + w, y), GR_dict[1], -1)  # Background for text
            cv2.putText(img, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        except Exception as e:
            print(f"Error during prediction: {e}")

    # Check if the image is valid before displaying
    if img is None or img.size == 0:
        print("Invalid image")
        continue  # Skip this iteration and try again

    # Show the live video feed
    cv2.imshow('LIVE', img)

    # Exit the loop if the ESC key is pressed
    key = cv2.waitKey(1)
    if key == 27:  # ESC key to break the loop
        cap.release()
        cv2.destroyAllWindows()
        break

    i += 1  # Increment the counter

# Display the full list of predictions
print("All predictions:", output)

# Find the most frequent predicted emotion using mode
try:
    final_output1 = st.mode(output)
    print("Most frequent emotion:", final_output1)
except st.StatisticsError:
    print("No mode found (possibly all predictions are unique)")

# Release the video capture and close any OpenCV windows
cap.release()
cv2.destroyAllWindows()


No faces detected
No faces detected
No faces detected
No faces detected
No faces detected
No faces detected
No faces detected
No faces detected
No faces detected
Input shape before prediction: (1, 96, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Input shape before prediction: (1, 96, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Input shape before prediction: (1, 96, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Input shape before prediction: (1, 96, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Input shape before prediction: (1, 96, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Input shape before prediction: (1, 96, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Input shape before prediction: (1, 96, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Input shape before prediction: (1, 96, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Input shape before prediction: (1, 96, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Input shape before prediction: (1, 96, 96, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


In [8]:
import statistics as st
final_output = st.mode(output)
final_output

'happy'